In [1]:
import os

from report_util import *
from trial import extract

WARMUP_ITERATIONS = 5
reports_dir = '/home/katie/Downloads/galette/slurm-1190029/'
pd.set_option('display.max_rows', 100)
data = extract(reports_dir, os.path.join(reports_dir, 'performance.csv'))
# Drop warm-up iterations
data = data[data['iteration'] >= WARMUP_ITERATIONS]
data.head()

Searching for trials in /home/katie/Downloads/galette/slurm-1190029/.
	Found 264 trials.
Checking trials.
	Missing required files for {'id': '227'}.
	Missing required files for {'id': '87'}.
	Missing required files for {'id': '136'}.
	Missing required files for {'id': '77'}.
	Missing required files for {'id': '147'}.
	Missing required files for {'id': '230'}.
	Missing required files for {'id': '173'}.
	Missing required files for {'id': '157'}.
	Missing required files for {'id': '246'}.
	Missing required files for {'id': '146'}.
	Missing required files for {'id': '222'}.
	Missing required files for {'id': '91'}.
	Missing required files for {'id': '139'}.
	Missing required files for {'id': '224'}.
	Missing required files for {'id': '252'}.
	Missing required files for {'id': '113'}.
	Missing required files for {'id': '84'}.
	Missing required files for {'id': '239'}.
	Missing required files for {'id': '76'}.
	Missing required files for {'id': '59'}.
	Missing required files for {'id': '232'

,iteration,rss,elapsed_time,benchmark,tool,commit_sha,branch_name,remote_origin_url,id
5,5,3468568,7937,jython,galette,e4c41ca82468d7ba4fad4da1f39ff8bb8bb8946f,main,git@github.com:neu-se/phosphor2.git,208
6,6,3447292,8337,jython,galette,e4c41ca82468d7ba4fad4da1f39ff8bb8bb8946f,main,git@github.com:neu-se/phosphor2.git,208
7,7,3656520,7482,jython,galette,e4c41ca82468d7ba4fad4da1f39ff8bb8bb8946f,main,git@github.com:neu-se/phosphor2.git,208
8,8,3851780,7654,jython,galette,e4c41ca82468d7ba4fad4da1f39ff8bb8bb8946f,main,git@github.com:neu-se/phosphor2.git,208
9,9,3519952,7011,jython,galette,e4c41ca82468d7ba4fad4da1f39ff8bb8bb8946f,main,git@github.com:neu-se/phosphor2.git,208


In [2]:
import numpy as np


def create_table_frame(data, benchmark, y):
    selected = select(data, benchmark=benchmark)
    tools = sorted(data['tool'].unique())
    rows = []
    baseline_values = select(selected, tool='galette')[y]
    sig_level = compute_sig_level(3)
    for tool in tools:
        values = select(selected, tool=tool)[y]
        if len(values) > 0:
            p = mann_whitney(baseline_values, values)
            median = np.median(values)
            sig = 'color: red;' if p < sig_level else ''
            effect_size = a12(baseline_values, values)
            rows.append([tool, benchmark, median, p, sig, effect_size, len(values)])
        else:
            rows.append([tool, benchmark, np.NaN, np.NaN, np.NaN, np.NaN, 0])
    return pd.DataFrame(rows, columns=['tool', 'benchmark', 'med', 'p', 'sig', 'a12', 'n'])


def create_table(data, y):
    benchmarks = sorted(data['benchmark'].unique())
    return pd.concat(create_table_frame(data, benchmark, y) for benchmark in benchmarks)


def format_column_names(names):
    return (
        names[0].title().replace('-', '').replace('None', 'Baseline'),
        names[1]
    )


def create_performance_table(data, y, title):
    raw_table = create_table(data, y)
    raw_table['med'] = raw_table['med'].astype(pd.Int64Dtype())
    table = raw_table.pivot(index=['benchmark'], values=['med', 'p', 'a12'], columns=['tool']) \
        .reorder_levels(axis=1, order=['tool', None]) \
        .sort_index(axis=1) \
        .sort_index(axis=0) \
        .reindex(['galette', 'none', 'mirror-taint', 'phosphor'], axis=1, level=0) \
        .reindex(['med', 'p', 'a12'], axis=1, level=1)
    # Remove comparison of Galette against itself
    table = table.drop(columns=[('galette', 'p'), ('galette', 'a12')])
    table.index.names = [None for _ in table.index.names]
    table.index = table.index.map(str.title)
    table.columns = table.columns.map(format_column_names)
    table.columns.names = [None for _ in table.columns.names]
    return table.style.format(precision=3, na_rep='---') \
        .set_caption(title)

In [3]:
create_performance_table(data, 'rss', 'Memory Usage (kB).')

In [4]:
create_performance_table(data, 'elapsed_time', 'Execution Time (ms).')